In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from src.extra.model import ModelDistribution
from src.extra.pprint import print_model
from src.core.core import reset_model

from src.greedy.greedy import greedy_algorithm
from src.greedy.resource_allocation_policy import SumPercentage
from src.greedy.server_selection_policy import SumResources
from src.greedy.task_prioritisation import UtilityDeadlinePerResource

from src.optimal.fixed_optimal import fixed_optimal
from src.core.fixed_task import FixedTask, SumSpeedPowFixedAllocationPriority

In [3]:
def print_results(result, tasks, servers):
    print(f'Social welfare: {result.social_welfare:.3f}, Percentage social welfare: {result.percentage_social_welfare:.3f}, Percentage tasks: {result.percentage_tasks_allocated:.3f}\n')
    # for server in servers:
    #    storage_usage = sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity
    #    computational_usage = sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity
    #    bandwidth_usage = sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity
    #    print(f'{server.name} resouce usage - Stor: {storage_usage:.3f}, Comp: {computational_usage:.3f}, Band: {bandwidth_usage:.3f}')
    print(f'Server avg storage: {np.mean([sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity for server in servers]):3.2f}, '
          f'computation: {np.mean([sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity for server in servers]):3.2f}, '
          f'bandwidth: {np.mean([sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity for server in servers]):3.2f}')
    
    avg_loading_speed = np.mean([task.loading_speed for task in tasks if task.running_server])
    avg_compute_speed = np.mean([task.compute_speed for task in tasks if task.running_server])
    avg_sending_speed = np.mean([task.sending_speed for task in tasks if task.running_server])
    print(f'\nTask avg loading speed: {avg_loading_speed:3.2f}, compute speed: {avg_compute_speed:3.2f}, sending speed: {avg_sending_speed:3.2f}')
    avg_loading_time = np.mean([task.required_storage / task.loading_speed for task in tasks if task.running_server])
    avg_compute_time = np.mean([task.required_computation / task.compute_speed for task in tasks if task.running_server])
    avg_sending_time = np.mean([task.required_results_data / task.sending_speed for task in tasks if task.running_server])
    print(f'Task avg loading time: {avg_loading_time:3.2f}, compute time: {avg_compute_time:3.2f}, sending time: {avg_sending_time:3.2f}')
    

def avg_resource(tasks, attribute):
    return np.mean([getattr(task, attribute) for task in tasks])

In [24]:
alibaba_model = ModelDistribution('../alibaba.mdl', num_tasks=40, num_servers=10)
alibaba_model.storage_scaling = 500
alibaba_model.computational_scaling = 1
alibaba_model.results_data_scaling = 5

In [25]:
tasks, servers = alibaba_model.generate()
print('Tasks - ' + ', '.join([f"{var}: {avg_resource(tasks, var):3.2f}" for var in ["required_storage", "required_computation", "required_results_data"]]))

fixed_tasks = [FixedTask(task, SumSpeedPowFixedAllocationPriority()) for task in tasks]
print('\nFixed tasks - ' + ', '.join([f"{var}: {avg_resource(fixed_tasks, var):3.2f}" for var in ["required_storage", "required_computation", "required_results_data"]]))
print('Fixed tasks - ' + ', '.join([f"{var}: {avg_resource(fixed_tasks, var):3.2f}" for var in ["loading_speed", "compute_speed", "sending_speed"]]))

foreknowledge_fixed_tasks = [FixedTask(task, SumSpeedPowFixedAllocationPriority(), resource_foreknowledge=True) for task in tasks]
print('\nForeknowledge fixed tasks - ' + ', '.join([f"{var}: {avg_resource(foreknowledge_fixed_tasks, var):3.2f}" for var in ["required_storage", "required_computation", "required_results_data"]]))
print('Foreknowledge fixed tasks - ' + ', '.join([f"{var}: {avg_resource(foreknowledge_fixed_tasks, var):3.2f}" for var in ["loading_speed", "compute_speed", "sending_speed"]]))

# fig, axs = plt.subplots(1, 4, figsize=(15, 4))
# for ax, var in zip(axs.flatten(), ['required_storage', 'required_computation', 'required_results_data', 'deadline']):
#     ax.hist([getattr(task, var) for task in tasks], bins=20)
#     ax.set_title(var)

Tasks - required_storage: 61.20, required_computation: 632.73, required_results_data: 22.02

Fixed tasks - required_storage: 148.62, required_computation: 1877.50, required_results_data: 22.02
Fixed tasks - loading_speed: 39.45, compute_speed: 73.75, sending_speed: 9.68

Foreknowledge fixed tasks - required_storage: 61.20, required_computation: 632.73, required_results_data: 22.02
Foreknowledge fixed tasks - loading_speed: 12.68, compute_speed: 22.95, sending_speed: 9.68


In [26]:
result = greedy_algorithm(tasks, servers, UtilityDeadlinePerResource(), SumResources(), SumPercentage())
print_results(result, tasks, servers)

Social welfare: 414.720, Percentage social welfare: 0.933, Percentage tasks: 0.850

Server avg storage: 0.51, computation: 0.72, bandwidth: 0.90

Task avg loading speed: 6.03, compute speed: 19.62, sending speed: 3.68
Task avg loading time: 15.96, compute time: 22.06, sending time: 6.48


In [27]:
reset_model(tasks, servers)
result = fixed_optimal(fixed_tasks, servers, time_limit=4)
print_results(result, fixed_tasks, servers)

Social welfare: 220.460, Percentage social welfare: 0.496, Percentage tasks: 0.275

Server avg storage: 0.39, computation: 0.82, bandwidth: 0.62

Task avg loading speed: 16.73, compute speed: 68.18, sending speed: 5.45
Task avg loading time: 12.92, compute time: 38.36, sending time: 5.84


Fixed optimal different objective values - cplex: 220.46 and running task values: 220.45999999999998


In [28]:
reset_model(tasks, servers)
result = fixed_optimal(foreknowledge_fixed_tasks, servers, time_limit=4)
print_results(result, foreknowledge_fixed_tasks, servers)

Social welfare: 382.280, Percentage social welfare: 0.860, Percentage tasks: 0.600

Server avg storage: 0.35, computation: 0.34, bandwidth: 0.99

Task avg loading speed: 8.92, compute speed: 13.12, sending speed: 6.62
Task avg loading time: 9.92, compute time: 31.27, sending time: 4.10


Fixed optimal different objective values - cplex: 382.28 and running task values: 382.28000000000003
